# Wrangle OpenStreetMap Data Using SQL

*Feng Li*

*Jun 23, 2016*

## Part 1: Map Area 

**Phoenix, Arizona, US**: https://mapzen.com/data/metro-extracts/#phoenix-arizona

In [34]:
OSMFILE = 'phoenix_arizona.osm'

In [35]:
import mapparser
mapparser.count_tags(OSMFILE)

defaultdict(int,
            {'bounds': 1,
             'member': 20372,
             'nd': 3103883,
             'node': 2572316,
             'osm': 1,
             'relation': 2074,
             'tag': 1899440,
             'way': 320238})

The osm file is 591.5 MB, with over 6,000,000 top level tags. The 1,899,440 tags in the file are name-value pair, to define multiple attributes of nodes or ways.

## Part 2: Problems Encountered in the Map 

### 1. Key Type

Four tag key categories:
1. "lower", for tags that contain only lowercase letters and are valid
2. "lower_colon", for otherwise valid tags with a colon in their names
3. "problemchars", for tags with problematic characters
4. "other", for other tags that do not fall into the other three categories

In [36]:
import tagtype
tagtype.process_map(OSMFILE)

{'k': 'service area', 'v': '20 miles'}
{'k': 'service area', 'v': '20 miles'}


{'lower': 1079975, 'lower_colon': 791985, 'other': 27478, 'problemchars': 2}

Two tags contain problematic characters.

### 2. Users

In [41]:
import users
all_users = users.get_all_users(OSMFILE)
len(all_users)

1072

In [42]:
# sort the all_users reversely by contribution counts
import operator
sorted_users = sorted(all_users.items(), key=operator.itemgetter(1), reverse=True)
sorted_users[:10]

[('292665', 1067109),
 ('1330847', 318301),
 ('157922', 193539),
 ('183021', 114786),
 ('4111', 98347),
 ('1507362', 72021),
 ('147510', 64955),
 ('100209', 62625),
 ('465150', 46665),
 ('32890', 37644)]

The leading user has contributed more than 1 million entries, far ahead of the second user.

### 3. Street Type 

Since data is crowd sourced, there is no standard format for street names. With an expected street type list ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", "Way", "Trail", "Parkway", "Commons", "Circle", "Terrace", "Highway"], we get the following dictionary of streets with unexpected types. We can see the majority of these streets ending with numbers or abbreviations like 'Rd' and 'Ave'.

In [44]:
import audit
audit.audit_street(OSMFILE)

defaultdict(set,
            {'101': {'E. Warner Rd Suite 101', 'East Baseline Road Suite 101'},
             '102': {'North Power Road #102'},
             '103': {'E Baseline Rd #103'},
             '104-401': {'S. Superstition Mountain Road St 104-401'},
             '105': {'North 90th Street #105'},
             '108': {'W Elliot Rd #108'},
             '110': {'East Doubletree Ranch Road #110',
              'N Centennial Way, Suite 110'},
             '114-225': {'South Higley Road, Suite 114-225'},
             '117': {'N 51st Ave #117'},
             '119': {'N. Gilbert Road, #119'},
             '121': {'3514 N. Power Road, Ste. 121'},
             '122': {'E Baseline Rd Ste 122', 'S Alma School Rd #122'},
             '130': {'E. Marilyn Rd., Bldg. 4, Suite 130',
              'East Marilyn Road Building 4 Suite 130'},
             '1400-1532': {'N. Central Avenue, Suite 1400-1532'},
             '200': {'N Arboleda, Suite 200',
              'W Union Hills Dr., Building D, 

For Overabbreviated street names, we could use a mapping dictionay to update with full names.

In [ ]:
mapping = { "St": "Street",
            "St.": "Street",
            "street": "Street",
            "Ave": "Avenue",
            "Ave.": "Avenue",
            "Blvd": "Boulevard",
            "Blvd.": "Boulevard",
            "Boulavard": "Boulevard",
            "Rd": "Road",
            "Rd.": "Road",
            "RD": "Road",
            "Pl": "Place",
            "Pl.": "Place",
            "PKWY": "Parkway",
            "Pkwy": "Parkway",
            "Ln": "Lane",
            "Ln.": "Lane",
            "Dr": "Drive",
            "Dr.": "Drive"
            }

In [48]:
audit.fix_street(OSMFILE)

West Happy Valley Rd => West Happy Valley Road
E Warner Rd => E Warner Road
East Williamsfield Rd => East Williamsfield Road
N Hayden Rd => N Hayden Road
S Watson Rd => S Watson Road
W Camelback Rd => W Camelback Road
N Scottsdale Rd => N Scottsdale Road
E Baseline Rd => E Baseline Road
E. Brown RD => E. Brown Road
East Rio Salado Pkwy => East Rio Salado Parkway
South Wall street => South Wall Street
5810 Alameda Rd. => 5810 Alameda Road
N. Cave Creek Rd. => N. Cave Creek Road
West Happy Vally Rd. => West Happy Vally Road
E Havasu Pl => E Havasu Place
N. 19th Ave. => N. 19th Avenue
Florence Blvd. => Florence Boulevard
Apache Blvd. => Apache Boulevard
W 18th St => W 18th Street
W McNeil St => W McNeil Street
E. Frontier St => E. Frontier Street
W Grand Ave => W Grand Avenue
Noth Central Ave => Noth Central Avenue
E Terrace Ave => E Terrace Avenue
North Arizona Ave => North Arizona Avenue
E Garnet Ave => E Garnet Avenue
S Buena Vista Ave => S Buena Vista Avenue
S Central Ave => S Central

### 4. City

In [46]:
other_city = audit.audit_city(OSMFILE)
len(other_city)

70

In [47]:
print other_city

set(['El Mirage', 'Gila Bend', 'Coolidge', 'Maricopa', 'Phenix', 'Anthem', 'Fort McDowell', 'Golbert', 'Carefree', 'Sacaton', 'Glendale', 'Morristown', 'Glemdale', '2036 N. Gilbert Rd.', 'Surprise', 'Gilbert', 'Sun City Wesr', 'Goodyear', 'Chandler', 'Msa', 'Higley', 'Rio Verde', 'Phoenx', 'tEMPE', 'Tempe', 'Mesa, AZ', 'Luke AFB, Waddell', 'Laveen Village', 'Peoria', 'Apache Junction', 'Scottsdale', 'Sun City West', 'Queen Creek', 'Tonopah', 'Sun Lakes', 'Arlington', 'Vulture City', 'Mesa', 'Maricopaaricopa', 'chandler', 'Fountain Hills', 'Luke AFB', 'Laveen', 'tempe', '25', 'Mobile', 'peoria', 'Wickenburg', 'Litchfield Park', 'scottsdale', 'Paradise Valley', 'Gold Canyon', 'Paradise Valley, AZ', 'casa Grande', 'mesa', 'Tollenson', 'Avondale', 'Sun City', 'Tolleson', 'Cave Creek', 'San Tan Valley', 'sun City West', 'New River', 'Tohono Oodham', 'Buckeye', 'Casa Grande', 'Florence', 'CHANDLER', 'Wittmann', 'Queen Valley'])


Apart from Phoenix, there are 70 other recorded cities in the data. But we can see serious inconsistency among them. 

Phoenix consists of many big boroughs, and in some tags city name is replaced with neighbourhood names and borough names. For example,'Laveen', or 'Laveen Village', is an urban village within the city of Phoenix (Source: Wikipedia). And some names are misspelled. 'Tollenson' is actually 'Tolleson', 'Phenix' and 'Phoenx' are actually 'Phoenix'. There is even a mistake putting street '2036 N. Gilbert Rd.' in the city tag.

### 5. Postcode 

The post code data for reference is downloaded from: http://www.unitedstateszipcodes.org/zip-code-database/. As it suggests, the post code for Phoenix city is from 85001 to 85099.

In [49]:
other_postcode = audit.audit_postcode(OSMFILE)
print len(other_postcode)
print other_postcode[20:30]

There are 1085 long post codes.
172
['8', 'AZ 85310', 'AZ 85249', 'AZ 85008', '85395', 'AZ 85242', '85310', 'AZ 85206', '85331', '86122']


There are 1085 long post codes which contain more than the main 5-digit codes. After removing the trailing 4-digit area codes, we get 172 unique post codes which are not in the reference database, and some of them begin with state post code 'AZ'.

### 6. Phone Number 

In [50]:
all_phone = audit.audit_phone(OSMFILE)
print all_phone[:10]

['(480) 897-8080', '+623-930-2188', '+623-842-8616', '+602-973-4000', '+602-841-7445', '+602-841-0704', '602-449-3300', '+602-336-2000', '+602-242-0281', '+602-246-0699']


Different formats of phone number are found in the data, among which are (XXX) XXX-XXXX, XXX-XXX-XXXX, +XXX-XXX-XXXX,  1XXXXXXXXXX, +1 XXX XXX XXXX, +1 XXX-XXX-XXXX, +1 XXX.XXX.XXXX, +1 (XXX) XXX-XXXX and even 'Phone number (XXX) XXX-XXXX' or a list.

## Part 3: Preparing Data for Database

To prepare the data for SQL database, I'll parse the elements in the OSM XML file, transforming them from document format to tabular format using **data.py**. The results are 5 .csv files:
+ nodes: top level node attributes. 

    **id  lat  lon  user  uid  version  changeset  timestamp**
    

+ node_tags: secondary tag attributes of node. 

    **id  key  value  type**
    

+ ways: top level way attributes. 

    **id  user  uid  version  changeset  timestamp**
    

+ way_tags: secondary tag attributes of way. 

    **id  key  value  type**
    

+ way_nodes: nd child tag attributes of way.

    **id  node_id  position**

I create a database 'osm' in MySQL Workbench, and then import these csv files to this database. This is one of the SQL queries used to import data.

In [ ]:
query ='''
LOAD DATA LOCAL INFILE 'C:\\Users\\lifeng\\Desktop\\osm\\ways.csv' 
REPLACE INTO TABLE `osm`.`ways` 
FIELDS TERMINATED BY ',' 
OPTIONALLY ENCLOSED BY '"' 
ESCAPED BY '"' 
LINES TERMINATED BY '\r\n' 
IGNORE 1 LINES (`id`, `user`, `uid`, `version`, `changeset`, `timestamp`);
'''

## Part 4: Data Overview

Before exploring the data, let's learn some basic statistics about the data.

### Full Size 

**nodes.csv**: 214.236 KB

**nodes_tags.csv**: 10,506 KB

**ways.csv**: 19,075 KB

**ways_nodes.csv**: 75.096 KB

**ways_tags.csv**: 55,472 KB

### Number Of Nodes

In [ ]:
query ='''
SELECT COUNT(*) FROM nodes;
'''

There are 2,572,316 nodes in the data.

### Number Of Ways

In [ ]:
query ='''
SELECT COUNT(*) FROM ways;
'''

There are 320,238 ways.

### Number Of Unique Users

In [ ]:
query ='''
CREATE TABLE userid
SELECT user.uid
FROM 
  (SELECT uid FROM nodes
  UNION ALL 
  SELECT uid FROM ways) user
  
  
SELECT COUNT(DISTINCT uid)
FROM userid
'''

Due to the low speed of MySQL Workbench to execute too complex queries with large table, first I create a table userid to store all the user IDs. I use UNION ALL to select all uid from table nodes and table ways, and get 2,892,554 records.

Then I count the distinct uid, which is 1062. This number is less than the result we get from users.get_all_users(OSMFILE), which is 1072. I think this is the result of ignoring some problematic entries during the data wrangling.

### Cities Other Than Phoenix

In [ ]:
query = '''
SELECT value, COUNT(*) AS num
FROM 
    (SELECT *
    FROM nodes_tags
    WHERE nodes_tags.key = 'city') city
GROUP BY city.value
ORDER BY num DESC;
'''

Apart from 4433 recods with city name being 'Phoenix' , there are 1350 'Glendale', 231 'Mesa', 202 'Peoria', 165 'Scottsdale', etc.

### Number of Schools

In [ ]:
query = '''
SELECT COUNT(*) AS num
FROM nodes_tags
WHERE nodes_tags.value = 'school';
'''

There are 483 schools in this district.

## Part 5: In-depth Data Exploration

### Top 10 Contributing Users 

In [ ]:
query ='''
SELECT uid, COUNT(uid) AS num
FROM userid
GROUP BY uid
ORDER BY num DESC
LIMIT 10;
'''

Comparing with the result from users.get_all_users(OSMFILE), the top 10 user IDs are identical, but with some lower counts. Again I think it's the result of data cleaning.

### Number of Users With Only 1 Contribution

In [ ]:
query ='''
SELECT COUNT(*) 
FROM
    (SELECT uid, COUNT(*) AS num
     FROM userid
     GROUP BY uid
     HAVING num=1) user;

'''

We get 190 users who contribute only 1 entry, among the total 1062 uses.

###  Amenities By Count

In [ ]:
query ='''
SELECT value, COUNT(*) AS num
FROM nodes_tags
WHERE nodes_tags.key = 'amenity'
GROUP BY value
ORDER BY num DESC;
'''

We get the aminities with more than 100 occurances in descending order are: fast_food, fuel, restaurant, place_of_worship, school, bench, swimming_pool, bank, pharmacy, fountain, toilets, cafe, trailer_park.

### Most Popular Cuisines 

In [ ]:
query = '''
SELECT value, COUNT(*) AS num
FROM nodes_tags 
WHERE nodes_tags.key = 'cuisine'
GROUP BY value
ORDER BY num DESC;
'''

The most popular cuisines in descending order are: burger, sandwich, pizza, coffee_shop, mexican, american, italian, chinese, chicken, ice_cream, etc. The occurance of burger (188) is more than double than the second cuisines sandwich and pizza (83).

### Religion Practice

In [ ]:
query = '''
SELECT nodes_tags.value, COUNT(*) AS num
FROM nodes_tags 
    INNER JOIN 
        (SELECT DISTINCT(id) 
        FROM nodes_tags 
        WHERE value='place_of_worship') place
    ON nodes_tags.id=place.id
WHERE nodes_tags.key='religion'
GROUP BY nodes_tags.value
ORDER BY num DESC;
'''

The difference here is huge, with 564 records of christian, 3 jewish, 2 bahai, 1 buddhist, 1 eckankar, 1 muslim.

### Types Of Highway

In [ ]:
query = '''
SELECT value, COUNT(*) as num
FROM ways_tags
WHERE ways_tags.key = 'highway'
GROUP BY value
ORDER BY num DESC;
'''

Residential tops the highway type list, followed by service, footway, secondary, tertiary, unclassified, track, primary, motorway_link, motorway, construction, cycleway, etc.

### Node Count Of Ways

In [ ]:
query = '''
SELECT COUNT(*)
FROM 
    (SELECT DISTINCT id, COUNT(*) as num
    FROM ways_nodes
    GROUP BY id
    HAVING num>1000) wn;
'''

There are 32 ways which have more than 1000 nodes, 145 ways with more than 500 nodes.

### Zip Code Of Ways

In [ ]:
query = '''
SELECT DISTINCT value FROM ways_tags
WHERE ways_tags.key LIKE 'zip%';
'''

Zip codes are stored in ways_tags table with column name 'zip_left' 'zip_right' 'zip_left1' 'zip_right2', so I use LIKE 'zip%' pattern. We know that zip code in Phoenix is between 85001 and 85099. But the result we get contains many zipcodes starting with '852' or '853'.

### Data Source

In [ ]:
query = '''
CREATE TABLE source
SELECT value, COUNT(*) AS num
FROM
((SELECT value FROM nodes_tags
WHERE nodes_tags.key = 'source'
GROUP BY value)
UNION ALL
(SELECT value FROM ways_tags
WHERE ways_tags.key = 'source'
GROUP BY value)) source
GROUP BY source.value
'''

First I create a table source to store all the source values from nodes_tags table and ways_tags table.

In [ ]:
query = '''
SELECT SUM(num) AS Bing
FROM
    (SELECT * FROM source
    WHERE value LIKE 'Bing%' OR value LIKE '%bing%') b
'''

There are 46 records from Wikipedia, 25 from Bing, 21 from Tiger, 17 from GPS,12 from Google, 10 from Yahoo.

## Part 6: Additional Ideas

**Ideas For Improvements**

In this report, data cleaning is far from enough. For example, the zip code field contains lists, and city field has 36 values other than the expected 'Phoenix'. The audit and fix functions are not applied when importing the data into csv files.

And there also should be official standards about the format and style of the input entries. For example, as for the city and zip code, user can only select from restricted options.

**Application of OSM**

If the user generated data have standards to follow and auditing tools to improve accuracy and consistancy, the OSM data could be put into broad application. Apart from navigation, people can use it to help with analysis and decision making. For example, businessmen can pick a location for starting a cafe where there are not enough cafes.

To encourage users to contribute data, easy-to-use web widgets could be created. For example, when adding a location on Facebook, users can be prompt to add some more information about it.